In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm
import pandas as pd
import csv
import re

<p style= "font-size:15px;font-weight:bold">Importing to pandas Dataframe</p>

In [2]:
df = pd.read_excel(r"C:\Users\770430\Desktop\Biogen_NPI_Files\Medpro\Input\medpro_hcp_input.xlsx", dtype=str)
df

,NPI_no,First_name,Last_name,City,State,Zip
0,1225181969,LINDA,NaN,NaN,NH,NaN
1,NaN,BRIDGET,TIERNEY,NaN,MN,55082
2,1295732089,JAMES,NaN,ORLANDO,NaN,32803
3,NaN,RYAN,MORGAN,NaN,KS,67037


In [3]:
df.fillna(value='', inplace=True)
df

,NPI_no,First_name,Last_name,City,State,Zip
0,1225181969,LINDA,,,NH,
1,,BRIDGET,TIERNEY,,MN,55082
2,1295732089,JAMES,,ORLANDO,,32803
3,,RYAN,MORGAN,,KS,67037


<p style= "font-size: 15px">Initialize <b>Dictionary</b> containing all the field names to be fetched</p>

In [4]:
SLN_DICT = {"MedProID":0, "Name":1, "Address_1":2, "Address_2":3, "Zip":4, "RDI":5, "License State":6, "License":7,
            "License Expires":8, "Status":9, "Prof Designation":10, "Certification Code":11, "Description":12}
len_SLN_DICT = len(SLN_DICT)
print(len_SLN_DICT)

DEA_DICT = {"MedProID":0, "Name":1, "Address_1":2, "Address_2":3, "Zip":4, "RDI":5, "DEA State":6, "DEA":7, 
            "DEA License Status":8, "DEA  Expires":9, "Registrant Type":10, "Business Activity Code":11, "Drug Schedule":12, 
            "Payment Indicator":13, "State Provided License":14, "Last Received Date":15}
len_DEA_DICT = len(DEA_DICT)
print(len_DEA_DICT)

CDS_DICT = {"MedProID":0, "Name":1, "Address_1":2, "Address_2":3, "Zip":4, "RDI":5, "CDS State":6, "CDS":7,
            "CDS License Status":8, "CDS  Expires":9, "CDS Prof Designation":10, "CDS Drug Schedule":11, 
            "CDS Specialty":12, "Disciplinary Action":13, "State Provided Prof Designation":14, 
            "State Provided License":15, "Last Received Date":16}
len_CDS_DICT = len(CDS_DICT)
print(len_CDS_DICT)

NPI_DICT = {"MedProID":0, "Name":1, "Mailing_Address":2,"Mail_Zip":3, "RDI":0, "Phone":0, "Fax":0, "Business_Address":7,
           "Bus_Zip":8, "NPI":12, "NPI State":13, "NPI Provided Status":14, "NPI Taxonomy":15, "NPI Provided State License":16,
           "Gender":17, "Enumeration Date":18, "Last Update Date":19, "CMS VPL Name":20,
           "CMS VPL State of License":21, "CMS VPL State License Number":22, 
           "State Provided License":23, "Last Received Date":24}
len_NPI_DICT = len(NPI_DICT)
print(len_NPI_DICT)

MPC_DICT = {"MPC NPI":0, "PAC ID":1, "Professional EnrollmentID":2, "Name":3, "Designation":4, "Group Practice":5, 
           "Group Practice PAC ID":6, "Address_1":7, "Address_2":8, "Zip":9, "RDI":10, "Phone":11, "Primary Specialty":12, 
           "Medical School":13, "Graduation Year":14, "First Received Date":15, "Last Received Date":16}
len_MPC_DICT = len(MPC_DICT)
print(len_MPC_DICT)

AMA_DICT = {"MedProID":0, "Name":1, "Address_1":2, "Address_2":3, "Zip":4, "RDI":5, "Telephone":6, "Medical Education":7, 
           "AMA Provided License State":8, "AMA Provided State License":9, "AMA Provided License Expires":10,
           "AMA Provided Status":11, "AMA PDRP":12, "AMA Contact Restrictions":13, "Prof Designation":14,
           "Primary Specialty":15, "Secondary Specialty":16, "Birth Date":17, "Birth CityCountry":18, "Graduation School":19,
           "Year of Graduation":20, "State Provided License":21, "As of Date":22, "Last Received Date":23}
len_AMA_DICT = len(AMA_DICT)
print(len_AMA_DICT)

BESTINFO_SET = {"MedProMasterID","HCP Profile Status","HCP Profile Designation","Veteran Administration HCP",
                "Confidence Level","CityStateZip","Primary","Secondary"}
len_BESTINFO_SET = len(BESTINFO_SET)
print(len_BESTINFO_SET)

13
16
17
22
17
24
8


<p style= "font-size: 15px">Initialize list containing all the column names to be included in the <b>csv</b> file</p>

In [5]:
OUTPUT_COLS = [ # --- SLN cols ---
                "SLN_MedProID","SLN_Name","SLN_Addr_Line_1","SLN_Addr_Line_2","SLN_City_State_Zip","SLN_RDI",
                "SLN_License_State","SLN_License_num","SLN_License_Expires","SLN_Status","SLN_Prof_Designation",
                "SLN_Certification_Code","SLN_Description",
                # --- DEA cols ---
                "DEA_MedProID","DEA_Name","DEA_Addr_Line_1","DEA_Addr_Line_2","DEA_City_State_Zip","DEA_RDI","DEA_State",
                "DEA_num","DEA_License_Status","DEA_Expires","Registrant_Type","Business_Activity_Code","Drug_Schedule",
                "Payment_Indicator","DEA_State_Provided_License","Last_Received_Date", 
                # --- CDS cols ---
                "CDS_MedProID","CDS_Name","CDS_Addr_Line_1","CDS_Addr_Line_2","CDS_City_State_Zip","CDS_RDI","CDS_State",
                "CDS_num","CDS_License_Status","CDS_Expires","CDS_Prof_Designation","CDS_Drug_Schedule","CDS_Specialty",
                "Disciplinary_Action","State_Provided_Prof_Designation","State_Provided_License","Last_Received_Date",
                # --- NPI cols ---
                "NPI_MedProID","NPI_Name","NPI_Mailing_Addr","NPI_Mailing_City_State_Zip","NPI_Mailing_RDI","NPI_Mailing_Phone",
                "NPI_Mailing_Fax","NPI_Bussiness_Addr","NPI_Business_City_State_Zip","NPI_Business_RDI","NPI_Business_Phone",
                "NPI_Business_Fax","NPI_num","NPI_State","NPI_Provided_Status","NPI_Taxonomy","NPI_Provided_State_License",
                "Gender","Enumeration_Date","Last_Update_Date","CMS_VPL_Name","CMS_VPL_State_of_License",
                "CMS_VPL_State_License_Number","NPI_State_Provided_License","Last_Received_Date",
                # --- MPC cols ---
                "MPC_NPI_num","MPC_PAC_ID","MPC_Professional_EnrollmentID","MPC_Name","Designation","Group_Practice",
                "Group_Practice_PAC_ID","MPC_Addr_Line_1","MPC_Addr_Line_2","MPC_City_State_Zip","MPC_RDI","MPC_Phone",
                "Primary_Specialty","Medical_School","Graduation_Year","First_Received_Date","Last_Received_Date",
                # --- AMA cols ---
                "AMA_MedProID","AMA_Name","AMA_Addr_Line_1","AMA_Addr_Line_2","AMA_City_State_Zip","AMA_RDI","AMA_Telephone",
                "Medical_Education","AMA_Provided_License_State","AMA_Provided_State_License","AMA_Provided_License_Expires",
                "AMA_Provided_Status","AMA_PDRP","AMA_Contact_Restrictions","Prof_Designation","Primary_Specialty",
                "Secondary_Specialty","Birth_Date","Birth_City_Country","Graduation_School","Year_of_Graduation",
                "State_Provided_License","As_of_Date","Last_Received_Date",
                # --- BestInfo cols ---
                "BInfo_MedProMasterID","HCP_Profile_Status","HCP_Profile_Designation","Veteran_Administration_HCP",
                "Confidence_Level","City_State_Zip","LL_Primary","LL_Secondary","PL_Primary","PL_Secondary"
              ]

<p style= "font-size: 15px">Write the header columns to <b>csv</b> file</p>

In [6]:
with open(r"C:\Users\770430\Desktop\Biogen_NPI_Files\Medpro\Output\medpro_hcp_write.csv", "w", newline= '') as csv_out:
    writer = csv.DictWriter(csv_out, fieldnames = OUTPUT_COLS)
    writer.writeheader()

<p style= "font-size: 15px">Define Medpro username and password</p>

In [7]:
MEDPRO_USER = "Biogen1"
MEDPRO_PWD  = "NewPass2020!"

<p style= "font-size: 15px">Define Chrome path and Medpro URL</p>

In [8]:
CROME_PATH = (r"C:\Users\770430\Anaconda3\chromedriver.exe")
MEDPRO_URL = "https://www.medproid.com/WebID.asp?action=clear"

In [9]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")
chrome_options.proxy = None

<p style= "font-size: 15px">Function to fetch <b>SLN</b> details of a particular entity</p>

In [10]:
def getPrimary_SLN():
    rows = driver.find_element_by_css_selector("div#DispSlnDetail tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    sln_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spSlnAddrLine1Std")
            addr2 = r.find_elements_by_id("spSlnAddrLine2Std")
            cszip = r.find_elements_by_id("spSlnCityStateZip")

            if (addr1):
                while (len(sln_list)<SLN_DICT["Address_1"]):
                    sln_list.append("")
                sln_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(sln_list)<SLN_DICT["Address_2"]):
                    sln_list.append("")
                sln_list.append(addr2[0].get_attribute("textContent").strip())
            elif (cszip):
                while (len(sln_list)<SLN_DICT["Zip"]):
                    sln_list.append("")
                sln_list.append(cszip[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in SLN_DICT):
                    while (len(sln_list) < SLN_DICT[content_title]):
                        sln_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        sln_list.append(content.strip())
                    else:
                        sln_list.append(content.strip())
    
    n = len(sln_list)
    #print(n)
    while (n < len_SLN_DICT):
        sln_list.append("")
        n += 1
        
    return sln_list

<p style= "font-size: 15px">Function to fetch <b>DEA</b> details of a particular entity</p>

In [11]:
def getPrimary_DEA():
    rows = driver.find_element_by_css_selector("div#DispDeaDetail tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    dea_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spDeaAddrLine1Std")
            addr2 = r.find_elements_by_id("spDeaAddrLine2Std")
            cszip = r.find_elements_by_id("spDeaCityStateZip")
            
            if (addr1):
                while (len(dea_list)<DEA_DICT["Address_1"]):
                    dea_list.append("")
                dea_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(dea_list)<DEA_DICT["Address_2"]):
                    dea_list.append("")
                dea_list.append(addr2[0].get_attribute("textContent".strip()))
            elif (cszip):
                while (len(dea_list)<DEA_DICT["Zip"]):
                    dea_list.append("")
                dea_list.append(cszip[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in DEA_DICT):
                    while (len(dea_list) < DEA_DICT[content_title]):
                        dea_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        dea_list.append(content.strip())
                    else:
                        dea_list.append(content.strip())
    
    n = len(dea_list)
    #print(n)
    while (n < len_DEA_DICT):
        dea_list.append("")
        n += 1
                
    return dea_list

<p style= "font-size: 15px">Function to fetch <b>CDS</b> details of a particular entity</p>

In [12]:
def getPrimary_CDS():
    rows = driver.find_element_by_css_selector("div#DispCdsDetail tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    cds_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spCdsAddrLine1Std")
            addr2 = r.find_elements_by_id("spCdsAddrLine2Std")
            cszip = r.find_elements_by_id("spCdsCityStateZip")
            
            if (addr1):
                while (len(cds_list)<CDS_DICT["Address_1"]):
                    cds_list.append("")
                cds_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(cds_list)<CDS_DICT["Address_2"]):
                    cds_list.append("")
                cds_list.append(addr2[0].get_attribute("textContent".strip()))
            elif (cszip):
                while (len(cds_list)<CDS_DICT["Zip"]):
                    cds_list.append("")
                cds_list.append(cszip[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in CDS_DICT):
                    while (len(cds_list) < CDS_DICT[content_title]):
                        cds_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        cds_list.append(content.strip())
                    else:
                        cds_list.append(content.strip())
    
    n = len(cds_list)
    #print(n)
    while (n < len_CDS_DICT):
        cds_list.append("")
        n += 1
                        
    return cds_list

<p style= "font-size: 15px">Function to fetch <b>NPI</b> details of a particular entity</p>

In [13]:
def updtNPI_Dict(rdi, phone, fax):
        NPI_DICT["RDI"] = rdi
        NPI_DICT["Phone"] = phone
        NPI_DICT["Fax"] = fax
        #print(NPI_DICT)

def getPrimary_NPI():
    rows = driver.find_element_by_css_selector("div#DispNpiDetail tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    npi_list = []
        
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spNpi2AddrLine1Std")
            cszip1 = r.find_elements_by_id("spNpi2CityStateZip")
            addr2 = r.find_elements_by_id("spNpiAddrLine1Std")
            cszip2 = r.find_elements_by_id("spNpiCityStateZip")
            
            if (addr1):
                while (len(npi_list)<NPI_DICT["Mailing_Address"]):
                    npi_list.append("")
                updtNPI_Dict(4,5,6)         # Call func to update 'rdi','phone','fax' values in NPI Dictionary
                npi_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(npi_list)<NPI_DICT["Business_Address"]):
                    npi_list.append("")
                updtNPI_Dict(9,10,11)      # Call func to update 'rdi','phone','fax' values in NPI Dictionary
                npi_list.append(addr2[0].get_attribute("textContent".strip()))
            elif (cszip1):
                while (len(npi_list)<NPI_DICT["Mail_Zip"]):
                    npi_list.append("")
                npi_list.append(cszip1[0].get_attribute("textContent").strip())
            elif (cszip2):
                while (len(npi_list)<NPI_DICT["Bus_Zip"]):
                    npi_list.append("")
                npi_list.append(cszip2[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in NPI_DICT):
                    while (len(npi_list) < NPI_DICT[content_title]):
                        npi_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        npi_list.append(content.strip())
                    else:
                        npi_list.append(content.strip())

    n = len(npi_list)
    #print(n)
    while (n < len_NPI_DICT+3):
        npi_list.append("")
        n += 1
        
    updtNPI_Dict(0,0,0)                
    return npi_list

<p style= "font-size: 15px">Function to fetch <b>MPC</b> details of a particular entity</p>

In [14]:
def getPrimary_MPC():
    rows = driver.find_element_by_css_selector("div#DispMedicare tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    mpc_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spMpcAddrLine1")
            addr2 = r.find_elements_by_id("spMpcAddrLine2")
            cszip = r.find_elements_by_id("spMpcCityStateZip")
            
            if (addr1):
                while (len(mpc_list)<MPC_DICT["Address_1"]):
                    mpc_list.append("")
                mpc_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(mpc_list)<MPC_DICT["Address_2"]):
                    mpc_list.append("")
                mpc_list.append(addr2[0].get_attribute("textContent".strip()))
            elif (cszip):
                while (len(mpc_list)<MPC_DICT["Zip"]):
                    mpc_list.append("")
                mpc_list.append(cszip[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in MPC_DICT):
                    while (len(mpc_list) < MPC_DICT[content_title]):
                        mpc_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        mpc_list.append(content.strip())
                    else:
                        mpc_list.append(content.strip())
    
    n = len(mpc_list)
    #print(n)
    while (n < len_MPC_DICT):
        mpc_list.append("")
        n += 1
                        
    return mpc_list

<p style= "font-size: 15px">Function to fetch <b>AMA</b> details of a particular entity</p>

In [15]:
def getPrimary_AMA():
    rows = driver.find_element_by_css_selector("div#DispAmaDetail tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    remove_hist_text = "View History"
    remove_view_text = "View All"
    ama_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            addr1 = r.find_elements_by_id("spAmaAddrLine1")
            addr2 = r.find_elements_by_id("spAmaAddrLine2")
            cszip = r.find_elements_by_id("spAmaCityStateZip")
            
            if (addr1):
                while (len(ama_list)<AMA_DICT["Address_1"]):
                    ama_list.append("")
                ama_list.append(addr1[0].get_attribute("textContent").strip())
            elif (addr2):
                while (len(ama_list)<AMA_DICT["Address_2"]):
                    ama_list.append("")
                ama_list.append(addr2[0].get_attribute("textContent".strip()))
            elif (cszip):
                while (len(ama_list)<AMA_DICT["Zip"]):
                    ama_list.append("")
                ama_list.append(cszip[0].get_attribute("textContent").strip())
            else:
                content_title = re.sub("[1-9-#:/]", '', cell[0].get_attribute("textContent")).strip()
                content_title = re.sub("(of$)", '', content_title).strip()
                if (content_title in AMA_DICT):
                    while (len(ama_list) < AMA_DICT[content_title]):
                        ama_list.append("")
                    content = cell[1].get_attribute("textContent").strip()
                    if (remove_hist_text in content or remove_view_text in content):
                        content = re.sub((remove_hist_text+'|'+remove_view_text), '', content)
                        ama_list.append(content.strip())
                    else:
                        ama_list.append(content.strip())
    
    n = len(ama_list)
    #print(n)
    while (n < len_AMA_DICT):
        ama_list.append('')
        n += 1
                        
    return ama_list

<p style= "font-size: 15px">Function to fetch <b>BestInfo</b> details of a particular entity</p>

In [16]:
def getPrimary_BestInfo():
    rows = driver.find_element_by_css_selector("div#DispBestInfo tbody").find_elements_by_tag_name("tr")
    #print(len(rows))
    bestInfo_list = []
    for r in rows:
        cell = r.find_elements_by_css_selector("td")
        if (len(cell)>1):
            content_title = cell[0].get_attribute("textContent")
            if (re.sub("[-#:/]", '', content_title).strip() in BESTINFO_SET):
                content = cell[1].get_attribute("textContent").strip()
                bestInfo_list.append(content.strip())
    
    n = len(bestInfo_list)
    #print(n)
    if (n < (len_BESTINFO_SET+2)):
        while (n < (len_BESTINFO_SET+2)):
            bestInfo_list.append('')
            n += 1
                        
    return bestInfo_list

<h3>Driver function to write data into csv file</h3>

In [17]:
def writeData_to_csv():
    with open(r"C:\Users\770430\Desktop\Biogen_NPI_Files\Medpro\Output\medpro_hcp_write.csv", "a", newline= '') as app_csv:
        writer = csv.writer(app_csv)
        rows = driver.find_element_by_id("tblHeader").find_elements_by_css_selector("tr.GridRowOdd")
        #print(len(rows))
        next_row = -1
        for _ in range(len(rows)+1):
            if (next_row>=0):
                curr_row[next_row].click()

            total_list = getPrimary_SLN() + getPrimary_DEA() + getPrimary_CDS() + getPrimary_NPI() + getPrimary_MPC() + getPrimary_AMA() + getPrimary_BestInfo()
            writer.writerow(total_list)
            curr_row = driver.find_element_by_id("tblHeader").find_elements_by_css_selector("tr.GridRowOdd")
            next_row += 1
            #print(next_row)

<p style= "font-size: 15px">Start the <b>webDriver</b></p>

In [18]:
driver = webdriver.Chrome(options=chrome_options, executable_path=CROME_PATH)
driver.get(MEDPRO_URL)

In [19]:
WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "loginform")))
driver.find_element_by_id("loginUsername").send_keys(MEDPRO_USER)
driver.find_element_by_id("loginPassword").send_keys(MEDPRO_PWD)
driver.find_element_by_id("btnLogin").click()
WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "td[onclick*='/WebID.asp']"))).click()

<p style= "font-size: 15px">Loop through each combination in the dataframe</p>

In [20]:
for elem in tqdm(range(len(df)), desc="Writing to file... "):
    #WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "quickSearch"))).send_keys(1225181969)
    driver.find_element_by_id("quickSearch").clear()
    driver.find_element_by_id("quickSearch").send_keys("{0} {1} {2} {3} {4} {5}".format(df.iloc[elem,0],df.iloc[elem,1],
                                                                                        df.iloc[elem,2],df.iloc[elem,3],
                                                                                        df.iloc[elem,4],df.iloc[elem,5]))
    WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "retrieveAllBtn"))).click()
    writeData_to_csv()
print("\nClosing Driver...",end="\r")
driver.quit()
print("Process Completed.")



Process Completed.


In [95]:
driver.quit()